In [ ]:
import sys
sys.path.append('../PETSIRD/python')
import numpy as np
import prd
from io import BytesIO
from prd.types import TimeBlock, CoincidenceEvent

# Generate fake data to convert to PETSIRD

Part 1: Python dtypes to PETSIRD format

In [9]:
N_events = 7 # Number of detected events
N_detectors = 10 # Number of detectors
counts = np.random.poisson(np.ones(N_events)*3)
detectors1 = np.random.choice(np.arange(N_detectors), N_events)
detectors2 = np.random.choice(np.arange(N_detectors), N_events)


In [10]:
events = []
for count, detector1, detector2 in zip(counts, detectors1, detectors2):
    for _ in range(count):
        events.append(CoincidenceEvent(detector_1_id=int(detector1), detector_2_id=int(detector2)))
time_block = TimeBlock(id=0, prompt_events=events)


In [11]:
with prd.BinaryPrdExperimentWriter('data.bin') as writer:
    writer.write_header(prd.Header())
    writer.write_time_blocks(
        (time_block,)
    )


In [12]:
with open('data.bin', 'rb') as f:
    data = f.read()
    with prd.BinaryPrdExperimentReader(BytesIO(data)) as reader:
        header = reader.read_header()
        time_blocks = [t for t in reader.read_time_blocks()]


Part 2: PETSIRD to python dtypes for recon

In [13]:
with open('data.bin', 'rb') as f:
    data = f.read()
    with prd.BinaryPrdExperimentReader(BytesIO(data)) as reader:
        header = reader.read_header()
        time_blocks = [t for t in reader.read_time_blocks()]


In [14]:
time_blocks


[TimeBlock(id=0, promptEvents=[CoincidenceEvent(detector1Id=5, detector2Id=2, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detector1Id=5, detector2Id=2, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detector1Id=5, detector2Id=2, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detector1Id=2, detector2Id=4, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detector1Id=2, detector2Id=4, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detector1Id=2, detector2Id=4, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detector1Id=7, detector2Id=5, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detector1Id=7, detector2Id=5, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detector1Id=2, detector2Id=3, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detector1Id=4, detector2Id=8, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detector1Id=4, detector2Id=8, tofIdx=0, energy1Idx=0, energy2Idx=0), CoincidenceEvent(detect